# dsdb.ConnectionManager

### What is it?
A connection manager is an object you will create before you connect to any dataset database. It will store, and manage the creation, backup, restoration, migration, and teardown of different databases in it's connection map.

### Why use it?
Besides checking the parameters of your connections for you. It helps make migration between databases really simple. Why would you do that? Let's say you ingested a dataset locally and want to push that dataset to a production or staging dataset database. With this, that is a simple call to get either all data or a subset of data from your local dsdb instance to the production or staging instance.

### Index
1) [Initialization](#Initialization)

2) Connect

3) Backup

4) Teardown

5) Restore

6) Migrate

## Initialization
Below are all the functions needed to create links to various instances of dataset databases.

**Note: This does not create the databases themselves this just creates links to them.**

In [1]:
import datasetdatabase as dsdb
user = "jacksonb"

In [2]:
mngr = dsdb.ConnectionManager(user=user)

In [3]:
# initialization a connection to a local database
mngr.add_connections(dsdb.LOCAL)
print(mngr)

LOCAL:
	driver: sqlite
	link: /active/examples/local_database/local.db


In [4]:
# pass a custom dictionary config to the connection manager
config = {"local_2": {"driver": "sqlite", "database": "/active/examples/local_database/local_2.db"}}
mngr.add_connections(config)
mngr

LOCAL:
	driver: sqlite
	link: /active/examples/local_database/local.db
local_2:
	driver: sqlite
	link: /active/examples/local_database/local_2.db

In [5]:
# override an existing connection with new config settings
config = {dsdb.LOCAL: {"driver": "sqlite", "database": "/active/examples/local_database/remake.db"}}
mngr.add_connections(config, override_existing=True)
mngr

LOCAL:
	driver: sqlite
	link: /active/examples/local_database/remake.db
local_2:
	driver: sqlite
	link: /active/examples/local_database/local_2.db

In [6]:
# initialize and pass connection settings
mngr = dsdb.ConnectionManager(dsdb.LOCAL, local_store="/active/examples/local_database/passed_path.db", user=user)
mngr

LOCAL:
	driver: sqlite
	link: /active/examples/local_database/passed_path.db

In [7]:
# muliple configs at once
config = {"prod": {
            "driver": "sqlite",
            "database": "/active/examples/local_database/prod.db"},
          "stg": {
            "driver": "sqlite",
            "database": "/active/examples/local_database/stg.db"}
         }

# can be passed directly into the ConnectionManager initialization function
mngr = dsdb.ConnectionManager(config, user=user)
mngr

prod:
	driver: sqlite
	link: /active/examples/local_database/prod.db
stg:
	driver: sqlite
	link: /active/examples/local_database/stg.db

In [8]:
# create a local instance with a custom link
mngr.add_connections(dsdb.LOCAL, local_store="/active/examples/local_database/custom_store.db")
mngr

prod:
	driver: sqlite
	link: /active/examples/local_database/prod.db
stg:
	driver: sqlite
	link: /active/examples/local_database/stg.db
LOCAL:
	driver: sqlite
	link: /active/examples/local_database/custom_store.db

In [9]:
# update a connection name
mngr.rename_connection(dsdb.LOCAL, "custom")
mngr

prod:
	driver: sqlite
	link: /active/examples/local_database/prod.db
stg:
	driver: sqlite
	link: /active/examples/local_database/stg.db
custom:
	driver: sqlite
	link: /active/examples/local_database/custom_store.db

In [10]:
# connection config can be retrieved using keys
mngr["custom"]

{'custom': {'driver': 'sqlite',
  'database': '/active/examples/local_database/custom_store.db'}}

In [11]:
# connection already exists
mngr.add_connections(dsdb.LOCAL)
mngr.add_connections(dsdb.LOCAL)
mngr

AssertionError: A connection with that name already exists.

In [12]:
# link already exists
mngr.add_connections({"custom": {"driver": "sqlite", "database": "/active/examples/local_database/custom_store.db"}})
mngr

AssertionError: A connection with that name already exists.

In [13]:
# missing driver
mngr.add_connections({"custom": {"database": "/active/examples/local_database/custom_store.db"}})
mngr

AssertionError: Connection requires a driver.

In [14]:
# missing link
mngr.add_connections({"custom": {"driver": "sqlite"}})
mngr

AssertionError: Connection requires a link.